# Chapter 9: Lifetimes

Learn how Rust ensures references are always valid.

## Understanding Lifetimes

Every reference in Rust has a lifetime - the scope for which that reference is valid.

Most of the time, lifetimes are implicit and inferred. We only need to annotate them when the compiler can't figure them out.

## Preventing Dangling References

In [ ]:
fn main() {
    // This won't compile
    // let r;
    // {
    //     let x = 5;
    //     r = &x;  // x doesn't live long enough
    // }
    // println!("r: {}", r);  // ERROR!
    
    // This works
    let x = 5;
    let r = &x;
    println!("r: {}", r);
}

## Generic Lifetimes in Functions

When a function returns a reference, Rust needs to know how long that reference is valid.

In [ ]:
// This won't compile without lifetime annotations
// fn longest(x: &str, y: &str) -> &str {
//     if x.len() > y.len() {
//         x
//     } else {
//         y
//     }
// }

// With lifetime annotations
fn longest<'a>(x: &'a str, y: &'a str) -> &'a str {
    if x.len() > y.len() {
        x
    } else {
        y
    }
}

fn main() {
    let string1 = String::from("long string is long");
    let string2 = String::from("xyz");
    
    let result = longest(string1.as_str(), string2.as_str());
    println!("Longest: {}", result);
}

## Lifetime Annotation Syntax

- Lifetime annotations start with `'` (apostrophe)
- Names are usually short, like `'a`, `'b`, etc.
- They describe relationships between lifetimes of references

In [ ]:
// Single reference parameter
fn first_word<'a>(s: &'a str) -> &'a str {
    let bytes = s.as_bytes();
    
    for (i, &item) in bytes.iter().enumerate() {
        if item == b' ' {
            return &s[0..i];
        }
    }
    
    &s[..]
}

fn main() {
    let sentence = String::from("hello world");
    let word = first_word(&sentence);
    println!("First word: {}", word);
}

## Thinking in Terms of Lifetimes

In [ ]:
// This function always returns the first parameter
// So y doesn't need the same lifetime
fn longest<'a>(x: &'a str, y: &str) -> &'a str {
    x
}

// This won't compile - returning a reference to local value
// fn invalid<'a>(x: &str, y: &str) -> &'a str {
//     let result = String::from("long string");
//     result.as_str()  // ERROR! result doesn't live long enough
// }

// Solution: return an owned value
fn valid(x: &str, y: &str) -> String {
    String::from("long string")
}

## Lifetime Annotations in Struct Definitions

In [ ]:
struct ImportantExcerpt<'a> {
    part: &'a str,
}

fn main() {
    let novel = String::from("Call me Ishmael. Some years ago...");
    let first_sentence = novel.split('.').next().expect("Could not find a '.'");
    
    let i = ImportantExcerpt {
        part: first_sentence,
    };
    
    println!("Excerpt: {}", i.part);
}

## Lifetime Elision Rules

The compiler uses three rules to figure out lifetimes when not explicitly annotated:

1. Each parameter that is a reference gets its own lifetime
2. If there's exactly one input lifetime, it's assigned to all output lifetimes
3. If there are multiple input lifetimes, but one is `&self` or `&mut self`, the lifetime of `self` is assigned to all output lifetimes

In [ ]:
// Rule 2: one input lifetime
fn first_word(s: &str) -> &str {
    // Compiler infers: fn first_word<'a>(s: &'a str) -> &'a str
    &s[..]
}

// Rule 3: method with self
struct ImportantExcerpt<'a> {
    part: &'a str,
}

impl<'a> ImportantExcerpt<'a> {
    fn level(&self) -> i32 {
        3
    }
    
    fn announce_and_return_part(&self, announcement: &str) -> &str {
        // Return lifetime tied to self, not announcement
        println!("Attention please: {}", announcement);
        self.part
    }
}

## The Static Lifetime

`'static` means the reference can live for the entire program duration.

In [ ]:
fn main() {
    // String literals have 'static lifetime
    let s: &'static str = "I have a static lifetime.";
    println!("{}", s);
    
    // Be careful: most of the time you don't want 'static!
    // Often a sign you're trying to create a dangling reference
}

## Generic Type Parameters, Trait Bounds, and Lifetimes Together

In [ ]:
use std::fmt::Display;

fn longest_with_announcement<'a, T>(
    x: &'a str,
    y: &'a str,
    ann: T,
) -> &'a str
where
    T: Display,
{
    println!("Announcement! {}", ann);
    if x.len() > y.len() {
        x
    } else {
        y
    }
}

fn main() {
    let string1 = String::from("abcd");
    let string2 = "xyz";
    
    let result = longest_with_announcement(
        string1.as_str(),
        string2,
        "Today is someone's birthday!",
    );
    
    println!("Longest: {}", result);
}

## Exercises

1. Write a function that returns the longer of two string slices
2. Create a struct that holds two references with different lifetimes
3. Implement a method that returns a reference from a struct
4. Fix lifetime errors in broken code

In [ ]:
// Exercise 1: Longer string
fn longer<'a>(x: &'a str, y: &'a str) -> &'a str {
    // Your code here
    x
}

// Exercise 2: Struct with two references
struct TwoRefs<'a, 'b> {
    // Your fields here
}

// Exercise 3: Method returning reference
struct Container<'a> {
    data: &'a str,
}

impl<'a> Container<'a> {
    fn get_data(&self) -> &str {
        // Your code here
        self.data
    }
}

## Key Takeaways

- Lifetimes ensure references are always valid
- Lifetime annotations describe relationships between references
- The compiler often infers lifetimes (elision rules)
- Structs holding references need lifetime annotations
- `'static` means the reference lives for the entire program
- Lifetime annotations don't change how long references live